# Asbestos Audit:
### outputs:
* ASB audit
* Duplicate APNs in share point, Tetra forms, Smart Sheets
* ASB sample sent prior to ASB assessment
* ASB results prior to ASB sample sent date


## Downloads needed:
* 2 Smartsheets both Central and South
* 3- Tetrat forms: Chimney tips, Abatment form, and survey form
* ESL SQL tracker
* Asbestos tracker - Sharepoint


In [1]:
import pandas as pd
import glob
import numpy as np
import os
import smartsheet
import datetime

# Gets smartsheets from teh source using API.

## no download needed

In [2]:
county= ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']

file_ex = 'xlsx'
file_csv = '.csv'


# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API

Central.errors_as_exceptions(True)

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)

# # Download Smartrsheets to 
# Central.Sheets.get_sheet_as_excel(Central_id, download_path=r"C:\Users\jacque.trahan\Downloads")
# South.Sheets.get_sheet_as_excel(South_id, download_path=r"C:\Users\jacque.trahan\Downloads") 

# Function to read in column titles and row vales and make out data frames
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))

df_both = pd.concat([Central_SS,South_SS])

#  Smart Sheets
## Gets our most recent dowloaded files from the downloads folder

In [3]:
# # read in Smart hseets tracker
# path1 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
# load_it1 = glob.glob(path1 + file_ex)

# df1 = pd.read_excel(max(load_it1, key=os.path.getctime), parse_dates=True)

In [4]:
# # read in South Central Smart sheeets tracker
# path_s = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"
# load_it_s = glob.glob(path_s + file_ex)

# df_s = pd.read_excel(max(load_it_s, key=os.path.getctime), parse_dates=True)

### Concat central and south cental files

In [5]:
# df_both = pd.concat([df1, df_s])

In [6]:
# Read in asbestos tracker from share point
path2 = r"C:\Users\jacque.trahan\Downloads\Asbestos Tracker*"
load_it2 = glob.glob(path2 + file_csv)

df2 = pd.read_csv(max(load_it2, key=os.path.getctime))

In [7]:
# df2.info()

In [8]:
# read in ESL from SQL server
path3 = r"C:\Users\jacque.trahan\Downloads\Asbestos Sample Status*"
load_it3 = glob.glob(path3 + file_ex)
esl_final = pd.read_excel(max(load_it3, key=os.path.getctime), header=1)

Tetra forms has a key error that we have to open the file first then save it as a new file. 
Only then will it all me to read the file in pandas
* KeyError: "There is no item named 'xl / sharedStrings.xml' in the archive"

In [9]:
# df1 = pd.read_excel('Central Branch  trackers/Central Division 2021 Debris Removal Tracker.xlsx')
# df2 = pd.read_csv("../tetra forms ASB and SA/Asbestos Tracker.csv", parse_dates=True)
# we have to open the file then save it with a different name as an excel. Find a way to solve this
df3= pd.read_excel("../../tetra forms ASB and SA/ASB_TetraForms_survey1.xlsx")

# load in ESL with Concat
### No longer needed. But here for referacne on how to concat all files in a folder.
#### New way has master file and can just be read from downloads. 
* reads all files in a folder


In [10]:
# path = r"C:\Users\jacque.trahan\Desktop\Dixie Fires Audit\ESLs\Central"

# esl_final = pd.concat(map(pd.read_excel, glob.glob(path + '/*.xlsx')))




* Below is What the above is doing 
* The above way is faster
* no list

path = r"C:\Users\jacque.trahan\Desktop\Dixie Fires Audit\ESLs\Central"
esl_final= glob.glob(path + "/*.xlsx")

li=[]
for filename in all_files:
    df = pd.read_excel(filename, index_col=None)
    li.append(df)

esl_final = pd.concat(li)

In [11]:
smartsheets = df_both.copy()
asbestos = df2.copy()
tetratForms = df3.copy()

# Most important info

What I need to Check Daily is Smart Sheets vs ENV. Trackers
* Chimneys and neshaps match
* FINAL RESULTS vs Smart sheets – biggest concern

* ASB Assessment Date Match

* End of day numbers match

# Asbestos set up

In [12]:
# Take out all divisions other than central.
asbestos = asbestos[asbestos['Division'].isin(['Central Division'])]

###  Get APN counts from Asbetsos

In [13]:
# pd.set_option('display.max_rows',1000)

In [14]:
asbestos['APN'].value_counts().to_excel('Asbestos tracker APN Counts_Share Point-central.xlsx')

In [15]:
# changes date column from object to a date.
asbestos['Date Collected'] = pd.to_datetime(asbestos['Date Collected']).dt.date.fillna(0)
asbestos['Chimney Tip Date'] = pd.to_datetime(asbestos['Chimney Tip Date']).dt.date.fillna(0)
asbestos['Chimney Abatement Completed Date'] = pd.to_datetime(asbestos['Chimney Abatement Completed Date']).dt.date.fillna(0)
asbestos['Asbestos Abatement Completed Date'] = pd.to_datetime(asbestos['Asbestos Abatement Completed Date']).dt.date.fillna(0)
asbestos['Date Shipped or Driven'] = pd.to_datetime(asbestos['Date Shipped or Driven']).dt.date.fillna("0")

In [16]:
asbestos.columns = map(str.upper, asbestos.columns)

In [17]:
asbestos.rename(columns={'APN':'APN_ASB'}, inplace=True)

In [18]:
# Column section all upper case
asb_cols = asbestos[['APN_ASB', 'FINAL RESULTS', 'CHIMNEY', 'CHIMNEY TIP DATE', 'CHIMNEY FINDING',
                 'NESHAP WALLS',
                 'DATE COLLECTED', 'DATE SHIPPED OR DRIVEN', 
                 'SAMPLE QUANTITY', 'POINT COUNT NEEDED', 'POINT COUNT RESULTS',
                 'ASBESTOS ABATEMENT COMPLETED DATE', 'CHIMNEY ABATEMENT COMPLETED DATE',
                 'PLANNING NOTES', 'DIVISION']]

# Tetra Forms setup

### Tetra Forms sheet sample_log setup

In [19]:
# read in the sample log of the survey forms to get the correct sample count
TF = pd.read_excel("../../tetra forms ASB and SA/ASB_TetraForms_survey1.xlsx", sheet_name='sample_log')

In [20]:
# will grab the highest sample taken
max_samples = TF.groupby(['apn'])[['samplecounter']].max().reset_index()


## Tetra Forms chimney tip setup
Combine chimney tip database with sample log

In [21]:
# Read in the chuimney tip database
TF_tips = pd.read_excel('../../tetra forms ASB and SA/ASB_TetraForms_tips1.xlsx')

In [22]:
TF_tips = TF_tips[TF_tips['county'].isin(county)]

In [23]:
# columns needed
TF_tips= TF_tips[['apn', 'inspection_date']]

### Get Chimney Tip APN counts in asbestos

In [24]:
TF_tips['apn'].value_counts().to_excel('Asbestos Chimney tip APN counts.xlsx') # 1 for 1 last check all good

In [25]:
# rename for easy reading for later merges
TF_tips.rename(columns={'inspection_date': 'TIP INSPECTION_TF'}, inplace=True)

In [26]:
# Need to make this series a datetime and fill in 0'sfor blank spots
TF_tips['TIP INSPECTION_TF'] = pd.to_datetime(TF_tips['TIP INSPECTION_TF']).dt.date.fillna(0)

In [27]:
# merge tips and sample log databases
merge1 = max_samples.merge(TF_tips,
                          left_on='apn',
                          right_on='apn',
                          how='outer')

# Tetra Forms Abatment date setup
Combine Abatment date database with merge 1 (tips and max samples)

In [28]:
# abatment clearance form logs
TF_abate = pd.read_excel('../../tetra forms ASB and SA/ASB_TetraForms_abatment1.xlsx')

In [29]:
# chossing couty for only central division. Only way to seperate the divisions in Tetra forms
TF_abate = TF_abate[TF_abate['county'].isin(county)]

In [30]:
# columns needed
TF_abate = TF_abate[['apn', 'abatement_end']]

#### Get Tetra Forms Abatment APN counts in asbestos

In [31]:
TF_abate['apn'].value_counts().to_excel('TF Abatmanet APN Counts central.xlsx')

In [32]:
# Need to make this series a datetime and fill in 0'sfor blank spots
TF_abate['abatement_end'] = pd.to_datetime(TF_abate['abatement_end']).dt.date.fillna(0)

In [33]:
# rename for easy reading on later merges
TF_abate.rename(columns={'abatement_end': 'ABATEMENT END_TF'}, inplace=True)

In [34]:
merge2 = merge1.merge(TF_abate,
                     left_on='apn',
                     right_on='apn',
                     how='outer')

# Master Tetra Forms setup from all thre databases
* Chimney tips
* abatement dates
* asbestos survey info

In [35]:
# merges in tetra froms surveys with sample logs, tips, and abataemetn databses.
tetratForms = tetratForms.merge(merge2,
                 left_on='apn',
                 right_on='apn',
                 how='outer')

In [36]:
# chossing couty for only central division. Only way to seperate the divisions in Tetra forms
tetratForms = tetratForms[tetratForms['county'].isin(county)]

In [37]:
# columns needed
tetratForms = tetratForms[['apn', 'chimneys','walls','inspection_date', 'county','samplecounter','TIP INSPECTION_TF',
                           'ABATEMENT END_TF' ]]

###  TF Survey APN counts

In [38]:
tetratForms['apn'].value_counts().to_excel('Asbestos Survey TF_APN counts central.xlsx')

In [39]:
# change date object to date time format
tetratForms['inspection_date'] = pd.to_datetime(tetratForms['inspection_date']).dt.date.fillna(0)

In [40]:
# rename out columns for easier readin on merges
tetratForms.rename(columns={'inspection_date': 'survey inspection date'}, inplace=True)

In [41]:
# make ALL columnS upper CASE
tetratForms.columns =map(str.upper, tetratForms.columns)

In [42]:
tetratForms.rename(columns={'APN': 'APN_TF'}, inplace=True)

# Smart Sheet set up

### Get APN counts from Smart Sheets

In [43]:
smartsheets['APN_ROW Segment'].value_counts().to_excel('Central_SS Tracker APN counts.xlsx')

In [44]:
# smartsheets["Chimney Tipped"].fillna(0, inplace=True)


#### This is need becasue BUG: pd.to_datetime() throws if caching is on with Null-like arguments
* solution: pd.to_datetime(df.colx.fillna(pd.NaT), errors='coerce')

In [45]:
# smartsheets["Chimney Tipped"] = pd.to_datetime(smartsheets["Chimney Tipped"].fillna(pd.NaT), errors='coerce')
smartsheets["ASB Assessment"] = pd.to_datetime(smartsheets["ASB Assessment"].fillna(pd.NaT), errors='coerce')
smartsheets["ASB Abatement"] = pd.to_datetime(smartsheets["ASB Abatement"].fillna(pd.NaT), errors='coerce')
smartsheets['ASB Results Received'] = pd.to_datetime(smartsheets['ASB Results Received'].fillna(pd.NaT), errors='coerce')

In [46]:
# changes date column from object to a date. and fill na with 0's
smartsheets["ASB Assessment"] = smartsheets["ASB Assessment"].dt.date.fillna(0)
# smartsheets["Chimney Tipped"] = smartsheets["Chimney Tipped"].dt.date.fillna(0)
smartsheets["ASB Abatement"] = smartsheets["ASB Abatement"].dt.date.fillna(0)
smartsheets['ASB Results Received'] = smartsheets['ASB Results Received'].dt.date.fillna(0)

In [47]:
smartsheets.rename(columns={'ASB Assessment': 'Date Collected', 'Chimney Tipped':'CHIMNEY TIP DATE',
                            'County': 'SS_COUNTY'}, inplace=True)

In [48]:
smartsheets.columns = map(str.upper, smartsheets.columns)

In [49]:
# Column section normal uppercase
ss_cols = smartsheets[['APN_ROW SEGMENT','STREET #','STREET NAME','STRUCTURAL STATUS','SS_COUNTY',
                       'ASB RESULTS', 'ASB RESULTS RECEIVED', 
                     'CHIMNEY','CHIMNEY TIP DATE',
                     'NESHAP WALLS', 
                     "DATE COLLECTED",'ASB SUSPECTED',
                     'ASB ABATEMENT', 'DIVISION']]
ss_cols = ss_cols[ss_cols['DIVISION'].isin(['Central Division'])]

# Smasrt merge first w/ ASB

In [107]:
smasrt_ASB = ss_cols.merge(asb_cols,
                           left_on='APN_ROW SEGMENT',
                           right_on='APN_ASB',
                           how='outer',
                         suffixes=('_SS','_ASB'))

# smartsheet w/ ASB merge with Tetraform

In [51]:
SS_ASB_TF = smasrt_ASB.merge(tetratForms,
           left_on="APN_ROW SEGMENT",
           right_on='APN_TF',
           how='outer',
           suffixes=("_SS", "_TF"))

# ESL Set up

In [52]:
esl_final.columns = map(str.upper, esl_final.columns)

In [53]:
esl_final.rename(columns={'DATE SHIPPED':'DATE SHIPPED_esl'}, inplace=True)

In [54]:
esl_final = esl_final[esl_final['DIVISION'].isin(['Central Division'])]

In [55]:
exceedances = esl_final[['APN', 'BULK PLM EXCEEDANCES']]

In [56]:
esl_final = esl_final[['APN', 'DATE SHIPPED_esl','BULK PLM EXCEEDANCES', 'DIVISION']]

In [57]:
esl_final['DATE SHIPPED_esl'] = pd.to_datetime(esl_final['DATE SHIPPED_esl']).dt.date.fillna("0")

In [58]:
# This no longer works need to research group by changes today. 
# esl = esl_final.groupby(['APN'])[['DATE SHIPPED_esl']].max().reset_index()

In [59]:
esl = esl_final.groupby(['APN','DATE SHIPPED_esl']).max().reset_index()
esl = esl[['APN','DATE SHIPPED_esl']]

# Merge SS_ASB_TF Merge with ESL master

In [60]:
pre_final = exceedances.merge(esl,
            left_on="APN",
            right_on='APN',
            how='outer')

In [61]:
final = SS_ASB_TF.merge(pre_final,
                       left_on="APN_ROW SEGMENT",
                       right_on='APN',
                       how='outer')

In [99]:
final[final['APN_ASB'].isna()]

,APN_ROW SEGMENT,APN_ASB,APN_TF,STREET #,STREET NAME,STRUCTURAL STATUS,SS_COUNTY,DATE COLLECTED_SS,DATE COLLECTED_ASB,SURVEY INSPECTION DATE,...,CHIMNEY ABATEMENT COMPLETED DATE,ASB ABATEMENT,ASBESTOS ABATEMENT COMPLETED DATE,ABATEMENT END_TF,ASB ABATEMENT Date-M,NESHAP WALLS_SS,NESHAP WALLS_ASB,WALLS,Neshap Wall-M,DIVISION_SS
262,041-631-012,NaN,041-631-012,4828.0,Creekside Drive,Tree Operations in Progress,El Dorado,2022-03-01,0,2022-03-01,...,NaN,0,0,0,1,0,0,0,1,Central Division
610,041-882-024,NaN,041-882-024,5396.0,Wildberry Drive,Re-scrape Needed,El Dorado,2022-03-04,0,2022-03-04,...,NaN,0,0,0,1,0,0,0,1,Central Division
642,041-632-008,NaN,041-632-008,4937.0,Creekside Drive,Pending Soil Sample Results,El Dorado,2022-03-02,0,2022-03-02,...,NaN,0,0,0,1,Yes,0,Yes,0,Central Division
940,041-532-003,NaN,041-532-003,7262.0,Winding Way,Ready for Tree Operations,El Dorado,2022-03-01,0,2022-03-01,...,NaN,2022-03-09,0,2022-03-09,0,0,0,0,1,Central Division
1174,041-513-001,NaN,041-513-001,4909.0,Edgewood Circle,Ready for Erosion Control,El Dorado,2022-03-02,0,2022-03-02,...,NaN,0,0,0,1,0,0,0,1,Central Division
1203,041-620-030,NaN,NaN,041-620-030,Humbird Road,ROE Received,El Dorado,0,0,0,...,NaN,0,0,0,1,0,0,0,1,Central Division
1204,041-541-019,NaN,NaN,5101.0,Parkside Drive,ROE Received,El Dorado,0,0,0,...,NaN,0,0,0,1,0,0,0,1,Central Division


# set up columns for QC order

In [63]:
# change the ND/POS to spell out for easier matching in audits
final['FINAL RESULTS'] = final['FINAL RESULTS'].str.replace("ND", 'NEGATIVE').str.replace("POS","POSITIVE")

# Get hacky and force blanks to say 0
this is beacsue some APN are not in every tracker yet

## Smartsheets 


In [64]:
final["DATE COLLECTED_SS"] = final["DATE COLLECTED_SS"].fillna(0)

 **I think the column that you are converting is in UNIX timestamp format.**

**You should use unit='s'**
* research this

In [65]:
# final['CHIMNEY TIP DATE_SS'].dtype

In [66]:
final['CHIMNEY TIP DATE_SS'] = pd.to_datetime(final['CHIMNEY TIP DATE_SS'].fillna(pd.NaT), errors='coerce')

In [67]:
# final['CHIMNEY TIP DATE_SS'].value_counts()

In [68]:
final['CHIMNEY TIP DATE_SS'] = final['CHIMNEY TIP DATE_SS'].fillna(0)
final['ASB RESULTS'] = final['ASB RESULTS'].fillna("0")

final['CHIMNEY_SS'] = final['CHIMNEY_SS'].str.replace("No", '0')
final['CHIMNEY_SS'] = final['CHIMNEY_SS'].fillna("0")

final["ASB ABATEMENT"] = final["ASB ABATEMENT"].fillna(0)

final["NESHAP WALLS_SS"] = final["NESHAP WALLS_SS"].str.replace("No", "0").fillna("0")
final['ASB SUSPECTED'] = final['ASB SUSPECTED'].fillna("0")


## ASB Tracker


In [69]:
final['DATE COLLECTED_ASB'] = final['DATE COLLECTED_ASB'].fillna(0)
final['DATE SHIPPED OR DRIVEN'] = final['DATE SHIPPED OR DRIVEN'].fillna("0")

final['CHIMNEY TIP DATE_ASB'] = final['CHIMNEY TIP DATE_ASB'].fillna(0)

final['SAMPLE QUANTITY'] = final['SAMPLE QUANTITY'].fillna(0)

final['FINAL RESULTS'] = final['FINAL RESULTS'].fillna("0")
final['POINT COUNT RESULTS'] = final['POINT COUNT RESULTS'].fillna("0")

final['CHIMNEY_ASB'] = final['CHIMNEY_ASB'].str.replace("No", "0")
final['CHIMNEY_ASB'] = final['CHIMNEY_ASB'].fillna('0')

final["ASBESTOS ABATEMENT COMPLETED DATE"] = final["ASBESTOS ABATEMENT COMPLETED DATE"].fillna(0)

final["NESHAP WALLS_ASB"] = final["NESHAP WALLS_ASB"].str.replace("No", "0").fillna("0")


## Tetra Forms


In [70]:
final['TIP INSPECTION_TF'] = final['TIP INSPECTION_TF'].fillna(0)

final['SURVEY INSPECTION DATE'] = final['SURVEY INSPECTION DATE'].fillna(0)

final['ABATEMENT END_TF'] = final['ABATEMENT END_TF'].fillna(0)

final['SAMPLECOUNTER'] = final['SAMPLECOUNTER'].fillna(0)

final['CHIMNEYS'] = final['CHIMNEYS'].str.replace('No', "0").fillna("0")


final["ABATEMENT END_TF"] = final["ABATEMENT END_TF"].fillna(0)

final["WALLS"] = final["WALLS"].str.replace("No", "0").fillna("0")


# ESL fillna with 0's

In [71]:
final['DATE SHIPPED_esl'] = final['DATE SHIPPED_esl'].fillna("0")
final['DATE SHIPPED OR DRIVEN'] = final['DATE SHIPPED OR DRIVEN'].fillna("0")

In [75]:
final = final[["APN_ROW SEGMENT",'APN_ASB','APN_TF','STREET #','STREET NAME','STRUCTURAL STATUS','SS_COUNTY',
               "DATE COLLECTED_SS",'DATE COLLECTED_ASB','SURVEY INSPECTION DATE',
               'SAMPLE QUANTITY','SAMPLECOUNTER',
               'ASB SUSPECTED', 'DATE SHIPPED OR DRIVEN','DATE SHIPPED_esl',
               'ASB RESULTS','FINAL RESULTS','POINT COUNT NEEDED', 'POINT COUNT RESULTS', 'ASB RESULTS RECEIVED', 'BULK PLM EXCEEDANCES',
               'CHIMNEY_SS','CHIMNEY_ASB','CHIMNEYS',
               'CHIMNEY TIP DATE_SS','CHIMNEY TIP DATE_ASB','TIP INSPECTION_TF',
               'CHIMNEY FINDING', 'CHIMNEY ABATEMENT COMPLETED DATE',
               'ASB ABATEMENT','ASBESTOS ABATEMENT COMPLETED DATE', 'ABATEMENT END_TF',
               'NESHAP WALLS_SS','NESHAP WALLS_ASB','WALLS',
               'DIVISION_SS']]

In [ ]:
final = final.copy()

# If ASB is suspected in SS shipped should not be blank in ASB tracker

In [76]:
def shipping_dates(df, comp1, comp2, comp3):
    if ((df[comp1] != "0") & ((df[comp2] == "0") | (df[comp3] == "0"))):
        return 0
    else:
        return 1

In [77]:
final.insert(final.columns.get_loc('DATE SHIPPED_esl')+1, 'Blank Ship Date',
            final.apply(shipping_dates,comp1='ASB SUSPECTED', comp2='DATE SHIPPED OR DRIVEN', comp3='DATE SHIPPED_esl',axis=1))

# Automate the Excel QC process Function for all below process

In [78]:
def compare_cols(df, comp1, comp2, comp3=None):
    if comp3 == None:
        if df[comp1] == df[comp2]:
            return 1
        else:
            return 0
    elif ((df[comp1] == df[comp2]) & (df[comp2] == df[comp3])):
        return 1
    else:
        return 0

# Check date shipped in ASB and date shipped in ESL

In [79]:
final.insert(final.columns.get_loc('Blank Ship Date')+1, 'Check if ship dates Match',
             final.apply(compare_cols, comp1='DATE SHIPPED OR DRIVEN', comp2='DATE SHIPPED_esl', axis=1))

# check Date Collected

In [80]:
final.insert(final.columns.get_loc('SAMPLE QUANTITY')+1,'Date Collected - M',
             final.apply(compare_cols, comp1="DATE COLLECTED_SS", comp2='DATE COLLECTED_ASB', comp3='SURVEY INSPECTION DATE', axis=1))

# check Total Samples Collected Counts

In [81]:
final.insert(final.columns.get_loc('SAMPLECOUNTER')+1, "Total Samples Collected Counts-M", 
            final.apply(compare_cols, comp1='SAMPLE QUANTITY', comp2='SAMPLECOUNTER',axis=1))

In [82]:
# have to use in place here or it will turn everythig to 0 if we use df[cols] = df[col].fillna(0) Like above
final['Total Samples Collected Counts-M'].fillna(0, inplace=True)

<ipython-input-82-13d0fb5395ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Total Samples Collected Counts-M'].fillna(0, inplace=True)


# check Final Results

Not working correctly. leaves a blank if there is an NS in column. Solve it

In [83]:

final.insert(final.columns.get_loc('BULK PLM EXCEEDANCES')+1, "Final Results-M", 
            final.apply(compare_cols, comp1="ASB RESULTS", comp2="FINAL RESULTS", axis=1))

In [84]:
final["Final Results-M"].fillna(0,inplace=True)

<ipython-input-84-99aeed858780>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final["Final Results-M"].fillna(0,inplace=True)


# Check Chimney 

In [85]:
final.insert(final.columns.get_loc('CHIMNEYS')+1, 'Has Chimney-M',
            final.apply(compare_cols, comp1="CHIMNEY_SS", comp2="CHIMNEY_ASB", comp3="CHIMNEYS", axis=1))

In [86]:
final['Has Chimney-M'].fillna(0, inplace=True)

<ipython-input-86-138484d10180>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Has Chimney-M'].fillna(0, inplace=True)


# check CHIMNEY TIP DATE

In [87]:
final.insert(final.columns.get_loc('TIP INSPECTION_TF')+1, "Chimney Tip dates-M", 
            final.apply(compare_cols, comp1="CHIMNEY TIP DATE_SS", comp2="CHIMNEY TIP DATE_ASB", comp3="TIP INSPECTION_TF", axis=1))

<ipython-input-78-e0cb4f201c79>:7: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  elif ((df[comp1] == df[comp2]) & (df[comp2] == df[comp3])):


# check Abatements Dates

In [88]:
final.insert(final.columns.get_loc('ABATEMENT END_TF')+1, "ASB ABATEMENT Date-M",
            final.apply(compare_cols,
                        comp1="ASB ABATEMENT", comp2="ASBESTOS ABATEMENT COMPLETED DATE", comp3="ABATEMENT END_TF", axis=1))

# check NESHAPS

In [89]:
final.insert(final.columns.get_loc('WALLS')+1, "Neshap Wall-M",
            final.apply(compare_cols, comp1="NESHAP WALLS_SS", comp2="NESHAP WALLS_ASB", comp3="WALLS", axis=1))

# Highight columns

In [90]:
# # set match columns to a certain color:
# # SYNTAX - df.style.set_properties(**{'background-color': 'color wanted'}, subset=["Col A", 'Col b', 'etc.'])

df_style = final.style.set_properties(**{'background-color': '#ffffb3'},
                           subset=['Date Collected - M',"Neshap Wall-M","ASB ABATEMENT Date-M",
                                  "Chimney Tip dates-M",'Has Chimney-M',
                                   "Final Results-M","Total Samples Collected Counts-M",
                                   'Blank Ship Date', 'Check if ship dates Match'], inplace=True)

# Make Smart sheets APNs the index and put in excel

In [91]:
# final.set_index("APN_ROW SEGMENT", inplace=True)

In [92]:
df_style.to_excel("Central Dixie ASB Audit.xlsx")

# Get number of ssamples taken compared to number of samples positive.
### this is extra if I have time.

In [93]:
# final[['FINAL RESULTS']].value_counts().plot(kind='pie',
#                                            labels=['No Data', "Negitives", "Positives", "NS"],
#                                              colors = ['yellow', 'green', 'red', 'lightblue'],
#                                           autopct='%.2f',
#                                            fontsize=15,
#                                           figsize=(6,6),
#                                            title=' ASB Overview by %',
#                                             ylabel="")



In [94]:
# final[['ASB RESULTS']].value_counts().plot(kind='pie',
#                                            labels=['No Data', "Negitives", "Positives"],
#                                            colors = ['yellow', 'green', 'red'],
#                                           autopct='%.2f',
#                                            fontsize=15,
#                                           figsize=(6,6),
#                                            title='Smart Sheets ASB Overview by %',
#                                           ylabel="")

In [95]:
# plt.figure(figsize=(12,6))
# sns.countplot(data=final, x='SS_COUNTY', hue='ASB RESULTS')


In [96]:
# plt.figure(figsize=(12,6))
# sns.countplot(data=final, x='SS_COUNTY', hue='FINAL RESULTS')

# ASB sample sent prior to ASB assessment

In [97]:
asb_sample = df_both.copy()
asb_sample = asb_sample[['APN_ROW Segment','Street #', 'Street Name', 'County',
                         'ASB Assessment', 'ASB Sample Sent']]
asb_sample = asb_sample[(asb_sample['ASB Assessment'] > asb_sample['ASB Sample Sent'])]

asb_sample.set_index('APN_ROW Segment', inplace=True)
asb_sample.to_excel('ASB sample sent prior to ASB assessment.xlsx')

# ASB results prior to ASB sample sent date

In [98]:
asb_res = df_both.copy()
asb_res = asb_res[['APN_ROW Segment','Street #', 'Street Name', 'County',
                   'ASB Sample Sent','ASB Results Received', 'ASB Results']]
asb_res = asb_res[(asb_res['ASB Sample Sent'].isna()) & (asb_res['ASB Results'].notna())]

asb_res.set_index('APN_ROW Segment', inplace=True)
asb_res.to_excel('ASB results prior to ASB sample sent date.xlsx')